In [1]:
cd /Data1/AttnLSTM_exp2

/Data1/AttnLSTM_exp2


## Load Packages

In [2]:
import os 
import pathlib
import scipy.io as spio
from scipy.signal import resample
from omegaconf import OmegaConf

import torch 
import torch.nn as nn 
import torch.nn.functional as F

## Load Modules & Trained Model

In [3]:
# from exp_settings import get_config
from dataset import calculate_features
from model import MHA_LSTM_simpler

config_path = '/Data1/AttnLSTM_exp2/exps/MHA_LSTM_Semi/2025-01-17_21:15:30/config_result.yaml'
config = OmegaConf.load(config_path)

model = MHA_LSTM_simpler()

In [4]:
best_params_path = "/Data1/AttnLSTM_exp2/exps/MHA_LSTM_Semi/2025-01-17_21:15:30/Best_student.pt"

model.load_state_dict(torch.load(best_params_path))
model.eval()

MHA_LSTM_simpler(
  (rnn): LSTM(40, 60, num_layers=3, batch_first=True, dropout=0.1, bidirectional=True)
  (layer_norm1): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  (selfattn_layer): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=120, out_features=120, bias=True)
  )
  (frame_linear): Sequential(
    (0): Linear(in_features=120, out_features=40, bias=True)
    (1): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
  (murmur_linear): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
  )
)

## Change filepath to test your data.

In [5]:
example_filepath = '/Data2/murmur/test/73497_AV.wav'

### Prediction of example file

In [6]:
sampling_rate = config.data_preprocess.sampling_rate
threshold = config.exp_results.optim_threshold_stu # optimized threshold

sr, recording = spio.wavfile.read(example_filepath)

if sr != sampling_rate:
    num_samples = int(len(recording) * sampling_rate / sr)
    recording = resample(recording, num_samples)
recording = torch.as_tensor(recording.copy())
    
mel, _ = calculate_features(recording, sampling_rate)
mel = mel.unsqueeze(0)

_, murmur_pred = model(mel)
murmur_pred = F.softmax(murmur_pred, dim= -1)

murmur_probability = murmur_pred.squeeze(0)[1].detach().cpu().item()

if murmur_probability > threshold:
    print('Present')
else:
    print("Absent")

Present


In [7]:
from dataset import load_dataframes

df_dict = load_dataframes(config)
test_recording_df = df_dict["test_recording_df"]
test_recording_df.head()


Train with 751 patients, 2531 recording files.


,murmur_timing,rec_murmur_label,patient_murmur_label,outcome_label
recording,,,,
33151_AV,Holosystolic,Absent,Present,Abnormal
33151_PV,Holosystolic,Absent,Present,Abnormal
33151_TV,Holosystolic,Present,Present,Abnormal
33151_MV,Holosystolic,Present,Present,Abnormal
38337_AV,nan,Absent,Absent,Abnormal


In [8]:
# Successed to predcit 73497_AV has Murmur
test_recording_df.loc["73497_AV"]["rec_murmur_label"]

'Present'